In [76]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import datetime
import warnings
warnings.filterwarnings("ignore")
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

./casia-train-model.ipynb
./README.md
./prise_en_main/open-y-and-baselines.ipynb
./prise_en_main/compute-baseline-obs.ipynb
./prise_en_main/open-x-forecast-3d-train.ipynb
./prise_en_main/open-x-station.ipynb
./prise_en_main/open-x-forecast-2d-train.ipynb
./prise_en_main/open-mask-data.ipynb
./prise_en_main/open-x-forecast-3d-test.ipynb
./prise_en_main/open-x-forecast-2d-test.ipynb
./prise_en_main/.ipynb_checkpoints/open-x-station-checkpoint.ipynb
./prise_en_main/.ipynb_checkpoints/open-y-and-baselines-checkpoint.ipynb
./prise_en_main/.ipynb_checkpoints/compute-baseline-obs-checkpoint.ipynb
./prise_en_main/.ipynb_checkpoints/open-x-forecast-3d-train-checkpoint.ipynb
./data/Other/Other/stations_coordinates.csv
./data/Other/Other/NW_masks.nc
./data/Test/Test/Id_month_test.csv
./data/Test/Test/X_station_test.csv
./data/Test/Test/X_forecast/2D_arpege_119.nc
./data/Test/Test/X_forecast/arpege_3D_isobar_118.nc
./data/Test/Test/X_forecast/2D_arome_119.nc
./data/Test/Test/X_forecast/arpege_3D_h

### load X_train

In [2]:
first_date = datetime.datetime(2016,1,1)    
last_date = datetime.datetime(2017,12,31)

In [3]:
# coordonnées coords
coords_fname  = './data/Other/Other/stations_coordinates.csv'#'/kaggle/input/defi-ia-2022-test/Other/Other/stations_coordinates.csv'
coords = pd.read_csv(coords_fname)
### df_traint
fname = './data/Train/Train/X_station_train.csv'
train = pd.read_csv(fname,parse_dates=['date'],infer_datetime_format=True)
# data frame complet
#train= df_train.merge(coords, on=['number_sta'], how='left')

In [4]:
# reorganisation des données.
train['number_sta']=train['number_sta'].astype('category')

#sort by station, then by date 
train = train.sort_values(['number_sta','date'])

In [5]:

method_traitements = ["mean","min","max","std"]
dict_to_split = {"ff":method_traitements,"t":method_traitements,"td":method_traitements
                 ,"hu":method_traitements,"dd":method_traitements,"precip":"sum"}
Base_obs = train[{"number_sta","date","ff","t","td","hu","dd","precip"}].copy()
Base_obs.set_index('date',inplace = True)  

# type de traitement.


Base_obs_type = Base_obs.groupby('number_sta').resample('D').agg(dict_to_split)
Base_obs_type = Base_obs_type.reset_index(['date','number_sta'])
Base_obs_type['number_sta'] = Base_obs_type['number_sta'].astype('category') 

In [6]:
#Select the observations the day before 
Base_obs_type['baseline_obs'] = Base_obs_type['precip']["sum"].shift(1)
Base_obs_type = Base_obs_type.sort_values(by=["number_sta","date"])
#del Base_obs['precip']
#Base_obs = Base_obs.rename(columns={'baseline_obs':'precip'})

In [7]:
del Base_obs_type["precip"]

In [8]:
Base_obs_type

number_sta       date        ff                                  t  \
                                  mean   min    max       std        mean   
0        14066001 2016-01-01  3.913750  2.26   6.04  1.261548  280.333750   
1        14066001 2016-01-02  8.041250  4.22  11.52  2.125735  282.936667   
2        14066001 2016-01-03  5.430417  2.88   8.32  1.401284  281.101667   
3        14066001 2016-01-04  6.715417  5.09   9.04  1.143693  281.055000   
4        14066001 2016-01-05  5.957083  3.86   8.04  1.241792  281.255833   
...           ...        ...       ...   ...    ...       ...         ...   
183742   95690001 2017-12-26  7.699167  4.50  12.28  1.966402  279.041667   
183743   95690001 2017-12-27  8.134167  4.11  12.49  2.592839  277.274583   
183744   95690001 2017-12-28  3.172500  1.29   6.96  1.459256  275.019583   
183745   95690001 2017-12-29  6.341250  1.26  10.86  2.948950  277.452500   
183746   95690001 2017-12-30  8.529583  5.62  10.74  1.349546  285.330833   

                                  ...        td         hu              \
           min     max       std  ...       std       mean   min   max   
0       276.91  282.59  2.018238  ...  1.187006  88.591667  79.9  98.1   
1       280.65  284.37  1.012619  ...  0.970644  82.300000  71.8  92.0   
2       279.77  282.70  0.920801  ...  1.251278  86.604167  80.6  91.4   
3       279.64  283.42  1.366913  ...  0.530700  80.645833  71.9  87.4   
4       279.67  283.11  1.033323  ...  0.734533  82.750000  76.9  88.6   
...        ...     ...       ...  ...       ...        ...   ...   ...   
183742  277.38  280.63  1.008614  ...  0.635049  85.587500  79.6  92.3   
183743  274.72  279.51  1.363101  ...  1.975703  86.275000  71.7  93.0   
183744  273.10  278.44  1.570075  ...  0.955560  86.816667  76.7  93.4   
183745  273.70  280.80  2.731512  ...  1.946016  86.637500  73.1  95.8   
183746  280.47  287.09  2.115520  ...  1.735812  88.175000  79.4  94.9   

                          dd                          baseline_obs  
             std        mean    min    max        std               
0       7.165249  146.500000  123.0  200.0  21.241878          NaN  
1       6.789506  205.625000  138.0  241.0  29.331408          0.2  
2       3.240166  195.250000  160.0  252.0  26.294734          3.4  
3       5.057150  212.666667  200.0  234.0   7.833410         11.7  
4       3.314920  205.041667  194.0  213.0   5.120115          0.6  
...          ...         ...    ...    ...        ...          ...  
183742  3.404768  202.333333  170.0  259.0  28.574337          0.0  
183743  5.432531  251.500000  177.0  310.0  50.598161          2.4  
183744  5.403112  253.583333  221.0  295.0  18.572810          3.2  
183745  8.154157  211.500000  116.0  282.0  44.928929          0.0  
183746  5.395348  230.958333  182.0  256.0  19.720832          4.4  

[183747 rows x 23 columns]

In [9]:
date = first_date
dates = []
while date <= (last_date - datetime.timedelta(days=1)):
    date += datetime.timedelta(days=1)
    dates.append(date)

d_dates =  pd.DataFrame(dates, columns = ['date'])
d_dates['day_index'] = d_dates.index

In [79]:
y_f = Base_obs_type.merge(d_dates,how="right",on = ["date"])
y_f = y_f[y_f['date']!=last_date]
y_f['Id'] = y_f[list(y_f.columns)[1]].astype(str) + '_' + \
                  y_f['day_index'].astype(str) 

In [80]:
y_f

,date,"(number_sta, )","(date, )","(ff, mean)","(ff, min)","(ff, max)","(ff, std)","(t, mean)","(t, min)","(t, max)",...,"(hu, min)","(hu, max)","(hu, std)","(dd, mean)","(dd, min)","(dd, max)","(dd, std)","(baseline_obs, )",day_index,Id
0,2016-01-02,14066001,2016-01-02,8.041250,4.22,11.52,2.125735,282.936667,280.65,284.37,...,71.8,92.0,6.789506,205.625000,138.0,241.0,29.331408,0.2,0,14066001_0
1,2016-01-02,14126001,2016-01-02,NaN,NaN,NaN,NaN,281.972917,280.10,283.45,...,83.5,95.5,3.771316,NaN,NaN,NaN,NaN,0.5,0,14126001_0
2,2016-01-02,14137001,2016-01-02,9.521667,6.69,11.73,1.446560,282.402083,279.80,283.84,...,75.5,92.2,5.788814,215.958333,167.0,250.0,26.574717,0.0,0,14137001_0
3,2016-01-02,14216001,2016-01-02,6.755000,4.46,8.43,1.101019,281.785417,279.48,283.23,...,75.6,89.8,4.868084,211.500000,171.0,242.0,23.596241,0.6,0,14216001_0
4,2016-01-02,14296001,2016-01-02,NaN,NaN,NaN,NaN,279.995417,277.90,281.79,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2,0,14296001_0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183493,2017-12-30,86137003,2017-12-30,4.028333,2.74,5.80,0.907915,286.465417,284.11,287.94,...,79.7,95.9,5.852777,221.166667,181.0,250.0,22.107773,1.2,728,86137003_728
183494,2017-12-30,86165005,2017-12-30,5.950833,3.91,8.47,1.397625,285.988333,284.17,287.35,...,82.2,95.4,4.182085,239.291667,208.0,260.0,16.182328,1.2,728,86165005_728
183495,2017-12-30,86272002,2017-12-30,NaN,NaN,NaN,NaN,286.118750,283.75,287.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.6,728,86272002_728
183496,2017-12-30,91200002,2017-12-30,4.212500,2.40,5.52,0.853470,285.455000,281.29,287.30,...,83.1,96.3,4.280619,229.708333,194.0,242.0,13.851634,7.3,728,91200002_728


In [12]:
X_train = y_f.drop(["day_index",("number_sta","")],axis=1)
X_train["month"] = X_train[("date","")].dt.month
X_train["day"] = X_train[("date","")].dt.day

In [13]:
X_train.drop(["date",("date","")],axis=1,inplace=True)


### Load X_test

In [147]:
def load_X(fname,coords_fname) :
    coords = pd.read_csv(coords_fname)
    train = pd.read_csv(fname,parse_dates=['date'],infer_datetime_format=True)
    # reorganisation des données.
    train['number_sta']=train['number_sta'].astype('category')
    #sort by station, then by date 
    train = train.sort_values(['number_sta','date'])
    method_traitements = ["mean","min","max","std"]
    dict_to_split = {"ff":method_traitements,"t":method_traitements,"td":method_traitements
                     ,"hu":method_traitements,"dd":method_traitements,"precip":"sum"}
    Base_obs = train[{"number_sta","date","ff","t","td","hu","dd","precip"}].copy()
    Base_obs.set_index('date',inplace = True)  

    # type de traitement.
    Base_obs_type = Base_obs.groupby('number_sta').resample('D').agg(dict_to_split)
    Base_obs_type = Base_obs_type.reset_index(['date','number_sta'])
    Base_obs_type['number_sta'] = Base_obs_type['number_sta'].astype('category')   
    #Select the observations the day before 
    Base_obs_type['baseline_obs'] = Base_obs_type['precip']["sum"].shift(1)
    Base_obs_type = Base_obs_type.sort_values(by=["number_sta","date"])
    #del Base_obs['precip']
    #Base_obs = Base_obs.rename(columns={'baseline_obs':'precip'})
    del Base_obs_type["precip"]
    date = first_date
    dates = []
    while date <= (last_date - datetime.timedelta(days=1)):
        date += datetime.timedelta(days=1)
        dates.append(date)

    d_dates =  pd.DataFrame(dates, columns = ['date'])
    d_dates['day_index'] = d_dates.index
    y_f = Base_obs_type.merge(d_dates,how="right",on = ["date"])
    y_f = y_f[y_f['date']!=last_date]
    y_f['Id'] = y_f[list(y_f.columns)[1]].astype(str) + '_' + \
                      y_f['day_index'].astype(str) 
    X_train = y_f.drop(["day_index",("number_sta","")],axis=1)
    X_train["month"] = X_train[("date","")].dt.month
    X_train["day"] = X_train[("date","")].dt.day
    X_train.drop(["date",("date","")],axis=1,inplace=True)
    return X_train

In [150]:
pd.read_csv(fname)

,dd,hu,td,t,ff,precip,month,Id
0,NaN,NaN,NaN,278.35,NaN,NaN,12,14047002_277_4
1,NaN,NaN,NaN,278.40,NaN,0.0,12,14047002_277_5
2,NaN,NaN,NaN,279.01,NaN,0.0,12,14047002_277_6
3,NaN,NaN,NaN,279.66,NaN,0.0,12,14047002_277_7
4,NaN,NaN,NaN,279.99,NaN,0.0,12,14047002_277_8
...,...,...,...,...,...,...,...,...
2304797,190.0,82.8,277.00,279.74,10.62,0.0,12,95690001_176_19
2304798,195.0,84.2,277.44,279.93,11.86,0.0,12,95690001_176_20
2304799,199.0,85.7,277.95,280.21,11.77,0.0,12,95690001_176_21
2304800,198.0,85.3,278.25,280.58,10.16,0.0,12,95690001_176_22


In [148]:
fname = "./data/Test/Test/X_station_test.csv"
coords_fname = './data/Other/Other/stations_coordinates.csv'
X_test = load_X(fname,coords_fname)

ValueError: Missing column provided to 'parse_dates': 'date'

### Load Y

In [17]:
fname = './data/Train/Train/Y_train.csv'
param = 'Ground_truth'  #weather parameter name in the file ('Ground_truth' about Y and 'Prediction' about baseline)
Y_train= pd.read_csv(fname, parse_dates=['date'], infer_datetime_format=True)
Y_train['number_sta'] = Y_train['number_sta'].astype('category')

In [18]:
Y_train

,date,number_sta,Ground_truth,Id
0,2016-01-02,14066001,3.4,14066001_0
1,2016-01-02,14126001,0.5,14126001_0
2,2016-01-02,14137001,3.4,14137001_0
3,2016-01-02,14216001,4.0,14216001_0
4,2016-01-02,14296001,13.3,14296001_0
...,...,...,...,...
183742,2017-12-31,86137003,5.0,86137003_729
183743,2017-12-31,86165005,3.2,86165005_729
183744,2017-12-31,86272002,1.8,86272002_729
183745,2017-12-31,91200002,1.6,91200002_729


In [19]:
Y_train = Y_train[Y_train["date"]!=last_date].copy()

In [77]:
### modele simple
baseline = pd.read_csv("./data/Train/Train/Baselines/Baseline_observation_train.csv",parse_dates=['date'],infer_datetime_format=True)

In [81]:
baseline = baseline.merge(coords, on=['number_sta'], how='left')

In [82]:
baseline

,number_sta,date,Prediction,Id,lat,lon,height_sta
0,14066001,2016-01-02,0.2,14066001_0,49.334,-0.431,2.0
1,14126001,2016-01-02,0.5,14126001_0,49.145,0.042,125.0
2,14137001,2016-01-02,NaN,14137001_0,49.180,-0.456,67.0
3,14216001,2016-01-02,0.6,14216001_0,48.928,-0.149,155.0
4,14296001,2016-01-02,0.2,14296001_0,48.795,-1.037,336.0
...,...,...,...,...,...,...,...
183493,86137003,2017-12-30,1.2,86137003_728,47.035,0.098,96.0
183494,86165005,2017-12-30,1.2,86165005_728,46.412,0.841,153.0
183495,86272002,2017-12-30,1.6,86272002_728,46.839,0.457,120.0
183496,91200002,2017-12-30,7.3,91200002_728,48.526,1.993,116.0


In [84]:
baseline.drop(["number_sta","date","Id"],axis = 1 ,inplace = True)

In [90]:
x_full = baseline.dropna()
y_full = Y_train[Y_train.index.isin(x_full.index)].dropna()
x_full = baseline[baseline.index.isin(y_full.index)].dropna()
ximp = x_full.values
yimp = np.array(y_full["Ground_truth"])

### Application du modèle.

In [21]:
from sklearn.impute import KNNImputer

In [22]:
### réaliser une imputation des données manquantes.

X = X_train.drop("Id",axis=1).values
imputer = KNNImputer(n_neighbors=2)
imputer.fit(X)

KNNImputer(n_neighbors=2)

In [96]:
X_full = X_train.dropna()
Y_full = Y_train[Y_train.index.isin(X_full.index)].dropna()
X_full = X_train[X_train.index.isin(Y_full.index)].dropna()
Ximp = X_full.drop("Id",axis=1).values
Yimp = np.array(Y_full["Ground_truth"])

In [131]:
from sklearn.model_selection import train_test_split

In [132]:
X_train, X_test, y_train, y_test = train_test_split(
...     Ximp, Yimp, test_size=0.33, random_state=42)

### Fonction MAPE

In [120]:
yp = np.array([1,2,3,5,6])
yt = np.array([1.2,2.6,3.7,5.8,6.2])
def MAPE(yp,yt) :
    taille = len(yp)
    mape = 0
    for i in range(taille) :
        mape += abs((yt[i]-yp[i])/(yt[i]+1))
    mape *= 100/taille
    return mape

### Linear regression

In [68]:
from sklearn.linear_model import LinearRegression

In [141]:
reg = LinearRegression().fit(X_train, y_train)

In [142]:
reg.score(X_train, y_train)

0.23240890767167122

In [143]:
MAPE(reg.predict(X_train),y_train)

104.6640436654025

In [144]:
MAPE(reg.predict(X_test),y_test)

104.24008913023096

In [133]:
X_train.shape

(58097, 23)

### RandomForestRegression

In [124]:
from sklearn.ensemble import RandomForestRegressor

In [136]:
regr = RandomForestRegressor()
regr.fit(X_train, y_train)

RandomForestRegressor()

In [138]:
regr.score(X_train,y_train)

0.9292575384605449

In [139]:
MAPE(regr.predict(X_train),y_train)

24.727974834056045

In [140]:
MAPE(regr.predict(X_test),y_test)

66.79082068989443